<a href="https://colab.research.google.com/github/chinge55/Poem-Generator/blob/main/Pretrained_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [3]:
PATH =  "/content/drive/MyDrive/word2vec_models/nepali_embeddings_word2vec.txt"


In [4]:
from gensim.models import KeyedVectors


In [5]:
model_pretrained = KeyedVectors.load_word2vec_format(PATH, binary=False)


In [6]:
model_pretrained.most_similar_cosmul(positive = ['केटी', 'राजा'], negative = ['केटा'])

[('मुगल', 0.8069719672203064),
 ('महारानी', 0.7991781234741211),
 ('राजाहरू', 0.799119770526886),
 ('राजगद्दी', 0.7980359196662903),
 ('अधिराजकुमार', 0.7924796938896179),
 ('राजदरबार', 0.7917264103889465),
 ('महाराजा', 0.7911354303359985),
 ('सम्राट्', 0.7840874791145325),
 ('राजामहाराजा', 0.7802377343177795),
 ('चन्द्रशम्शेर', 0.7725237607955933)]

In [7]:
word_vectors = model.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [8]:
print("Number of word vectors:{}".format(len(word_vectors.vocab)))

Number of word vectors:515851


In [9]:
nb_words = len(word_vectors.vocab)
WV_DIM = word_vectors.vector_size

In [10]:
nb_words, WV_DIM

(515851, 300)

In [11]:
wv_matrix = (np.random.rand(nb_words, WV_DIM)-0.5)/5.0

Now use this word2vec values with all the values in the dataset and use the embeddings of the words present in the dataset and leave all the others as zeros

In [ ]:
for word, i in word_index.items():
  if i>= nb_words:
    continue
  try:
    embedding_vector = word_vectors[word]
    # words not found in embedding index will be all-zeros
    wv_matrix[i] = embedding_vector
  except:
    pass

NameError: ignored

In [12]:
!git clone https://github.com/devkotasawal1/Poem-Generator


Cloning into 'Poem-Generator'...
remote: Enumerating objects: 10, done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 10
Unpacking objects: 100% (10/10), done.


In [13]:
path_to_file = "Poem-Generator/lspd.txt"

In [70]:
from nltk.tokenize.api import StringTokenizer
tokenizer = StringTokenizer()

In [71]:
from collections import Counter
vocab = Counter

In [72]:
def text_to_wordlist(text):
  text = tokenizer.tokenize(text)
  vocab.update(text)
  return text 

In [73]:
def process_data(text_sentences):
  sentences = []
  for text in tqdm(text_sentences):
    txt = text_to_wordlist(text)
    sentences.append(txt)
  return sentences

In [107]:
data_file = open(path_to_file, 'r')
file_contents = data_file.read()
sentences_split = file_contents.splitlines()

In [109]:
len(sentences_split)

4091

In [99]:
import nltk
nltk.download('punkt') 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [147]:
vocab = Counter()
words_split = []
for i in range(len(sentences_split)):
  words = ""
  for idx, word in enumerate(sentences_split[i].split()):
    #words = words + " " +word
    word = word_tokenize(word)
    vocab.update(word)
    words_split.append(word)
  #words_split.append(words)

In [148]:
words_split[88]

['आँसुले']

In [138]:
len(vocab)

8572

In [143]:
vocab['मदन'], vocab['मुना']

(28, 35)

In [149]:
model_new = Word2Vec(words_split, size = 100, window = 5, min_count = 5, workers = 16, sg = 0, negative = 5)

In [152]:
model.wv.most_similar('जीव')

[('गरी', 0.30912837386131287),
 ('हेरेर', 0.2999993562698364),
 ('मेरा', 0.265689492225647),
 ('प्यारी', 0.25575417280197144),
 ('फर्क', 0.25408729910850525),
 ('भोटेको', 0.253245085477829),
 ('झन', 0.25235211849212646),
 ('मदन', 0.24645788967609406),
 ('हुँ', 0.24622337520122528),
 ('गर्नु', 0.24535807967185974)]

In [153]:
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200

In [154]:
from keras.preprocessing.sequence import pad_sequences

In [158]:
word_index = {t[0]:i+1 for i, t in enumerate(vocab.most_common(MAX_NB_WORDS))}

In [159]:
word_index

{'!': 1,
 ',': 2,
 '।': 3,
 '?': 4,
 'छ': 5,
 'यो': 6,
 'हो': 7,
 'म': 8,
 'ती': 9,
 'के': 10,
 'र': 11,
 'एक': 12,
 'हे': 13,
 'ए': 14,
 'तिमी': 15,
 'नै': 16,
 'त्यो': 17,
 'रे': 18,
 'न': 19,
 'छन्': 20,
 'मेरो': 21,
 'मेरी': 22,
 '“': 23,
 'आमा': 24,
 'तिम्रो': 25,
 'क्या': 26,
 'आँखा': 27,
 'मलाई': 28,
 '”': 29,
 '’': 30,
 'भो': 31,
 'कि': 32,
 'आत्मा': 33,
 'मुना': 34,
 'सब': 35,
 'नि': 36,
 'संसार': 37,
 'सारा': 38,
 'मन': 39,
 'जीवन': 40,
 '‘': 41,
 'फूल': 42,
 'घर': 43,
 'हामी': 44,
 'सुन्दर': 45,
 'मेरा': 46,
 'किन': 47,
 'झैँ': 48,
 'मदन': 49,
 'त': 50,
 'तर': 51,
 'हाम्रो': 52,
 'यी': 53,
 'हेर': 54,
 'आज': 55,
 'मनको': 56,
 'होला': 57,
 'भाइ': 58,
 'पनि': 59,
 'यस': 60,
 'कुन': 61,
 'जो': 62,
 'ईश्वर': 63,
 'फेरि': 64,
 'कसरी': 65,
 'आयो': 66,
 'ही': 67,
 'यहाँ': 68,
 'हृदय': 69,
 'मनमा': 70,
 'झैं': 71,
 'स्वर्ग': 72,
 'अब': 73,
 'मानिस': 74,
 'साथ': 75,
 'होस्': 76,
 'छौ': 77,
 'भै': 78,
 'बाबा': 79,
 'आँसु': 80,
 'दिन': 81,
 'पृथिवी': 82,
 'साथी': 83,
 'बादल': 84,
 'हात